<a href="https://colab.research.google.com/github/syma-afsha/AI_LAB/blob/main/practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Exam for Deep Network Development course. First part (mandatory)**

This notebook contains the task to be solved in order to pass the exam.
This is the first part of the exam, which is compolsury in order to get a grade. It contains a task similar to what you have worked on during the semester, which consists on implementing a network architecture and a function.

Please note that, to **PASS** the Deep Network Development course you must **SUBMIT A SUCCESSFUL SOLUTION FOR THE FIRST PART**. If you **FAIL** the first part, you have the right to do the exam **ONE MORE TIME**. If you **FAIL AGAIN**, then unfortunately, you have failed the course. If you **PASS** the first part, then you get the weighted average of your quizzes and assignments as your final grade.

## Your information
Please fill the next cell with your information

**Full Name**:

**Neptun code:**

**Date:** 16/01/2024 9AM-10AM

## Task Description

#### Your task is to implement a custom architecture layers and the forward functions.

#### Afterwards, make sure to run the last cell code to check if your implementation is correct.

#### This task should be **SOLVED IN 1 HOUR** and submitted to Canvas (download the .ipynb file). Please note that after 1 hour, the Canvas exam assignment will be closed and you cannot submit your solution.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
device = torch.device("cpu")
device

device(type='cpu')

# Architecture

Please right click the image and "Open image in a new tab" to view it better with zoom. Or download it from here: https://drive.google.com/file/d/1I4GNCq7OnANpLfknHFox52wnzlBnvmTj/view?usp=sharing

<br>
<br>

![](https://drive.google.com/uc?id=1I4GNCq7OnANpLfknHFox52wnzlBnvmTj)


# Text Encoder

In [ ]:
text = "Brown fox jumps over the lazy dog"

# Simple preprocessing the text
word_to_ix = {"Brown": 0, "fox": 1, "jumps": 2, "over": 3, "the": 4, "lazy": 5, "dog":6}
input_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long) # a tensor representing words by integers

In [ ]:
class TextEncoder(nn.Module):
    def __init__(self):
        super(TextEncoder, self).__init__()

        #IMPLEMENT LAYERS
        self.embedding=nn.Embedding(num_embeddings=7, embedding_dim=512)
        self.linear1=nn.Linear(in_features=3584, out_features=2048)
        self.linear2=nn.Linear(in_features=2048, out_features=12288)
        self.relu=nn.ReLU()


    def forward(self, text):
        #IMPLEMENT FORWARD STEP
        x=self.embedding(text)
        print(f"After embedding: {x.shape}")
        x=x.view(1,-1)
        print(f"After view: {x.shape}")
        x=self.linear1(x)
        print(f"After linear1: {x.shape}")
        x=self.relu(x)
        x=self.linear2(x)
        print(f"After linear2: {x.shape}")
        x=self.relu(x)


        # reshape embedding output to fit feed forward layer

        return x


encoder = TextEncoder()
text_encoder=encoder(input_tensor)

After embedding: torch.Size([7, 512])
After view: torch.Size([1, 3584])
After linear1: torch.Size([1, 2048])
After linear2: torch.Size([1, 12288])


# Combined Text-Image Encoder

In [ ]:
class CombinedTextImageEncoder(nn.Module):
    def __init__(self):
        super(CombinedTextImageEncoder, self).__init__()
        # IMPLEMENT LAYERS
        self.conv1=nn.Conv2d(in_channels=3, out_channels=26, kernel_size=3, padding=1)
        self.pool1=nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2=nn.Conv2d(in_channels=26, out_channels=40, kernel_size=3, stride=4,padding=1)
        self.linear1=nn.Linear(in_features=2560, out_features=4096) #change out_features
        self.relu=nn.ReLU()


    def forward(self, encoded_text):
      # IMPLEMENT FORWARD STEP
      print(f"Input shape: {encoded_text.shape}")
      # implement conv layers
      x=self.conv1(encoded_text)
      print(f"After conv1: {x.shape}")
      x=self.pool1(x)
      print(f"After pool1: {x.shape}")
      x=self.conv2(x)
      print(f"After conv2: {x.shape}")
      batch_size, channels, height, width = x.shape
      x=x.view(batch_size, -1)
      x=self.linear1(x)
      print(f"After linear1: {x.shape}")
      x=self.relu(x)


      #reshape output to fit Linear

      #add batch number to Linears output

      return x

#Combined Text-Image Decoder

In [ ]:
class CombinedTextImageDecoder(nn.Module):
    def __init__(self):
        super(CombinedTextImageDecoder, self).__init__()
        #IMPLEMENT LAYERS
        self.gru=nn.GRU(input_size=4096, hidden_size=256, num_layers=3, batch_first=True)
        self.conv1=nn.Conv2d(in_channels=4, out_channels=10, kernel_size=3, padding=0, stride=1)
        self.pool1=nn.AvgPool2d(kernel_size=2, stride=1)
        self.linear=nn.Linear(in_features=250, out_features=360)
        self.relu=nn.ReLU()
        self.convtrans1=nn.ConvTranspose2d(in_channels=20, out_channels=24, kernel_size=3, stride=3, padding=3)
        self.convtrans2=nn.ConvTranspose2d(in_channels=24, out_channels=48, kernel_size=4, stride=6, padding=3)
        self.convtrans3=nn.ConvTranspose2d(in_channels=48, out_channels=3, kernel_size=4, stride=4, padding=0) #out change




    def forward(self, features):
      print(f"Input shape: {features.shape}")
      # IMPLEMENT FORWARD PASS FOLLOW INSTRUCTIONS
      x1,_=self.gru(features)
      print(f"After GRU: {x1.shape}")
      x1=x1.view(1,-1)
      print(f"After view: {x1.shape}")
      x1=x1.unsqueeze(2).unsqueeze(3).view(1,4,8,8)
      print(f"After unsqueeze: {x1.shape}")

      x1=self.conv1(x1)
      print(f"After conv1: {x1.shape}")
      x=self.pool1(x1)
      print(f"After pool1: {x.shape}")
      x=x.view(1,-1)
      print(f"After view: {x.shape}")
      x=self.linear(x)
      print(f"After linear: {x.shape}")
      x=self.relu(x)
      x=x.unsqueeze(2).unsqueeze(3).view(1,10,6,6)
      print(f"After unsqueeze: {x.shape}")
      x=torch.concat((x1,x),dim=1)
      print(f"After concatenation: {x.shape}")
      x=self.convtrans1(x)
      print(f"After convtrans1: {x.shape}")
      x=self.convtrans2(x)
      print(f"After convtrans2: {x.shape}")
      x=self.convtrans3(x)
      print(f"After convtrans3: {x.shape}")
      x=self.relu(x)




      #reshape GRU output to fit conv2D

      #reshape Linear output to fit concatenation

      return x

# Combining everything

In [ ]:
class TextToImageModel(nn.Module):
    def __init__(self):
        super(TextToImageModel, self).__init__()
        self.textencoder = TextEncoder()
        self.encoder = CombinedTextImageEncoder()
        self.decoder = CombinedTextImageDecoder()
        #write the layer with the interrogation mark

    def forward(self, text):
      text_features = self.textencoder(text)
      print(f"Text features: {text_features.shape}")
      #making random noise image
      image = torch.rand(256, 256, 3).numpy()
      image = (image * 255).astype(np.uint8)
      image = np.rollaxis(image, 2, 0)
      input_image = torch.from_numpy(image) # Convert image to torch tensor
      input_image = torch.unsqueeze(input_image, dim=0) # Add a batch dimension of 1
      print(f"Input image: {input_image.shape}")
      image_reshape=input_image[:,:,0:64,0:64]
      print(f"Image reshape: {image_reshape.shape}")
      #reshape text_features to 4D tensor
      text_feature1=text_features
      text_feature1=text_feature1.unsqueeze(2).unsqueeze(3)
      print(f"Text features reshape: {text_feature1.shape}")
      text_feature1=text_feature1.view(1,3,64,64)
      print(f"Text features reshape: {text_feature1.shape}")

      x=image_reshape+text_feature1
      print(f"add: {x.shape}")
      x=self.encoder(x)
      print(f"Encoder: {x.shape}")
      x=x.unsqueeze(1)
      print(f"Unsqueeze: {x.shape}")

      x=self.decoder(x)
      print(f"Decoder: {x.shape}")




      # do the block with the interogation mark


      #reshape features to fit addition

      #write other layers


      return

model = TextToImageModel()
model(input_tensor)

After embedding: torch.Size([7, 512])
After view: torch.Size([1, 3584])
After linear1: torch.Size([1, 2048])
After linear2: torch.Size([1, 12288])
Text features: torch.Size([1, 12288])
Input image: torch.Size([1, 3, 256, 256])
Image reshape: torch.Size([1, 3, 64, 64])
Text features reshape: torch.Size([1, 12288, 1, 1])
Text features reshape: torch.Size([1, 3, 64, 64])
add: torch.Size([1, 3, 64, 64])
Input shape: torch.Size([1, 3, 64, 64])
After conv1: torch.Size([1, 26, 64, 64])
After pool1: torch.Size([1, 26, 32, 32])
After conv2: torch.Size([1, 40, 8, 8])
After linear1: torch.Size([1, 4096])
Encoder: torch.Size([1, 4096])
Unsqueeze: torch.Size([1, 1, 4096])
Input shape: torch.Size([1, 1, 4096])
After GRU: torch.Size([1, 1, 256])
After view: torch.Size([1, 256])
After unsqueeze: torch.Size([1, 4, 8, 8])
After conv1: torch.Size([1, 10, 6, 6])
After pool1: torch.Size([1, 10, 5, 5])
After view: torch.Size([1, 250])
After linear: torch.Size([1, 360])
After unsqueeze: torch.Size([1, 10, 6,

# Trying Out the whole architecture DO NOT MODIFY THESE CELLS!!!!


In [ ]:
text = "Brown fox jumps over the lazy dog"

# Simple preprocessing the text
word_to_ix = {"Brown": 0, "fox": 1, "jumps": 2, "over": 3, "the": 4, "lazy": 5, "dog":6}
input_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long) # a tensor representing words by integers

In [ ]:
model = TextToImageModel()
model = model.to(device)
output = model(input_tensor)



After embedding: torch.Size([7, 512])
After view: torch.Size([1, 3584])
After linear1: torch.Size([1, 2048])
After linear2: torch.Size([1, 12288])
Text features: torch.Size([1, 12288])
Input image: torch.Size([1, 3, 256, 256])
Image reshape: torch.Size([1, 3, 64, 64])
Text features reshape: torch.Size([1, 12288, 1, 1])
Text features reshape: torch.Size([1, 3, 64, 64])
add: torch.Size([1, 3, 64, 64])
Input shape: torch.Size([1, 3, 64, 64])
After conv1: torch.Size([1, 26, 64, 64])
After pool1: torch.Size([1, 26, 32, 32])
After conv2: torch.Size([1, 40, 8, 8])
After linear1: torch.Size([1, 4096])
Encoder: torch.Size([1, 4096])
Unsqueeze: torch.Size([1, 1, 4096])
Input shape: torch.Size([1, 1, 4096])
After GRU: torch.Size([1, 1, 256])
After view: torch.Size([1, 256])
After unsqueeze: torch.Size([1, 4, 8, 8])
After conv1: torch.Size([1, 10, 6, 6])
After pool1: torch.Size([1, 10, 5, 5])
After view: torch.Size([1, 250])
After linear: torch.Size([1, 360])
After unsqueeze: torch.Size([1, 10, 6,